# Machine Learning Zoomcamp - Homework 3

In [23]:
# Importing libraries

import urllib.request
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

filename = 'carprice_HW3.csv'

In [ ]:
# Downloading data

url='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'
urllib.request.urlretrieve(url, filename)

In [24]:
# Importing data
data=pd.read_csv(filename)
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [25]:
list_col=['Make', 'Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG',
          'city mpg','MSRP']
data_filtered=data[list_col]
data_filtered.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [26]:
data_filtered.columns = data_filtered.columns.str.replace(' ', '_').str.lower()
data_filtered=data_filtered.fillna(value=0)
data_filtered=data_filtered.rename(columns={'msrp':'price'})

In [27]:
data_filtered.isna().sum()>0

make                 False
model                False
year                 False
engine_hp            False
engine_cylinders     False
transmission_type    False
vehicle_style        False
highway_mpg          False
city_mpg             False
price                False
dtype: bool

---
### Q1: What is the most frequent observation (mode) for the column transmission_type?

In [28]:
data_filtered['transmission_type'].mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

---
### Q2: What are the two features that have the biggest correlation in this dataset?

In [29]:
numerical=['engine_hp','engine_cylinders','highway_mpg', 'city_mpg','year']
data_filtered[numerical].corr()

,engine_hp,engine_cylinders,highway_mpg,city_mpg,year
engine_hp,1.000000,0.774851,-0.415707,-0.424918,0.338714
engine_cylinders,0.774851,1.000000,-0.614541,-0.587306,-0.040708
highway_mpg,-0.415707,-0.614541,1.000000,0.886829,0.258240
city_mpg,-0.424918,-0.587306,0.886829,1.000000,0.198171
year,0.338714,-0.040708,0.258240,0.198171,1.000000


In [30]:
price_mean=data_filtered['price'].mean()
data_filtered['above_average']=np.where(data_filtered['price'] > price_mean,1,0)

In [31]:
data_filtered.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0


In [32]:
data_train_full, data_test=train_test_split(data_filtered, test_size=0.2,random_state=42)
data_train, data_val=train_test_split(data_train_full, test_size=0.2,random_state=42)

data_train=data_train.reset_index(drop=True)
data_test=data_test.reset_index(drop=True)
data_val=data_val.reset_index(drop=True)
y_train=data_train.above_average.values
y_val=data_val.above_average.values

In [33]:
del data_train['above_average']
del data_val['above_average']
del data_test['above_average']

### Q3: Which variable has the lowest mutual information score?

In [34]:
from sklearn.metrics import mutual_info_score
categorical=['make','model','transmission_type','vehicle_style']
def calculate_mi(series):
    return mutual_info_score(series, data_train_full.above_average)
round(data_train_full[categorical].apply(calculate_mi),2).sort_values()


transmission_type    0.02
vehicle_style        0.08
make                 0.24
model                0.46
dtype: float64

### Q4: Calculate the accuracy

In [35]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#one-hot encoding training data
data_train_dict=data_train[numerical+categorical].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(data_train_dict)
X_train= dv.transform(data_train_dict)

#fitting logistic regression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

#one-hot encoding validation data
data_val_dict=data_val[numerical+categorical].to_dict(orient='records')
X_val= dv.transform(data_val_dict)

y_pred=model.predict(X_val)

original_accuracy=np.round(accuracy_score(y_val,y_pred),2)
original_accuracy

0.94

### Q5: Which feature has the smallest difference?

In [36]:
all_features=numerical+categorical

for feature in all_features:
    subset=all_features.copy()
    subset.remove(feature)
    data_train_dict=data_train[subset].to_dict(orient='records')
    dv=DictVectorizer(sparse=False)
    dv.fit(data_train_dict)
    X_train= dv.transform(data_train_dict)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train,y_train)

    #one-hot encoding validation data
    data_val_dict=data_val[subset].to_dict(orient='records')
    X_val= dv.transform(data_val_dict)

    y_pred=model.predict(X_val)

    accuracy=np.round(accuracy_score(y_val,y_pred),2)
    print(f"Model without {feature} has an accuracy of {accuracy} which is {round(original_accuracy-accuracy,4)} different from the original accuracy")


Model without engine_hp has an accuracy of 0.93 which is 0.01 different from the original accuracy
Model without engine_cylinders has an accuracy of 0.94 which is 0.0 different from the original accuracy
Model without highway_mpg has an accuracy of 0.95 which is -0.01 different from the original accuracy
Model without city_mpg has an accuracy of 0.94 which is 0.0 different from the original accuracy
Model without year has an accuracy of 0.95 which is -0.01 different from the original accuracy
Model without make has an accuracy of 0.94 which is 0.0 different from the original accuracy
Model without model has an accuracy of 0.92 which is 0.02 different from the original accuracy
Model without transmission_type has an accuracy of 0.95 which is -0.01 different from the original accuracy
Model without vehicle_style has an accuracy of 0.93 which is 0.01 different from the original accuracy


### Q6: Which alpha leads to the best RMSE on the validation set?

In [37]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

data_filtered['price']=np.log1p(data_filtered['price'])

data_train_full, data_test = train_test_split(data_filtered, test_size=0.2, random_state=42)
data_train, data_val = train_test_split(data_train_full, test_size=0.25, random_state=42)

data_train = data_train.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)

y_train = data_train.price.values
y_val = data_val.price.values
y_test = data_test.price.values

del data_train['price']
del data_val['price']
del data_test['price']

train_dict = data_train[categorical + numerical].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)


val_dict = data_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = Ridge(alpha=0, solver="sag", random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
RMSE


0.48679431324238925

In [38]:
alpha_params=[0, 0.01, 0.1, 1, 10]
for a in alpha_params:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    RMSE = np.sqrt(mean_squared_error(y_val, y_pred))
    
    print(f"For alpha parameter ={a}, Ridge Regression model has an RMSE score of {round(RMSE, 3)}")

For alpha parameter =0, Ridge Regression model has an RMSE score of 0.487
For alpha parameter =0.01, Ridge Regression model has an RMSE score of 0.487
For alpha parameter =0.1, Ridge Regression model has an RMSE score of 0.487
For alpha parameter =1, Ridge Regression model has an RMSE score of 0.487
For alpha parameter =10, Ridge Regression model has an RMSE score of 0.487
